# Exploring 471 records from 1543 (Kickback from Lex's R Code)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 471 records (of the 1543 OTHER records) that were returned after being ran through Lex's R Code. The process for exploring these records is as follows:`

1. Explore the 'DATASET' field, which indicates what type of Origin Location Reference appears within the 'REMARKS' field for each record (e.g., AIRPORT, NAVAID, AIRPORT DIRECTLY OVER, NAVAID DIRECTLY OVER, RUNWAY, FIX RADIAL, HELIPORT, etc) 
<br>

2. For each of the unique Origin Location References that appear in the 'DATASET' field:

    a. Run the uas_loc_parts() function of the dataframe 
    
    b. Link to the appropriate Origin Location Reference linking function (e.g., link_airportsCleaned_ident(), link_airportsCleaned_icao(), nav_link())
        i. Separate the field that were successfully linked with the Origin Location Reference Dataset(s) and those that were not. Those that were not linked successfully will be moved to the exceptions file for further review.
    c. Run the successfully linked dataframe through the uas_lat_long() function to calculate the UAS Latitude/Longitude coordinates for each respective record/remark. 

***
### INPUT: 
*Dataframe (read from a .csv file) containing 471 records, of 1543 records that did not hit on the initial 'XX NM dir XXX' regular expressions, and was not initially picked up by our Runways regular expressions.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*Designated Point: 1 record (no link to Designated Points)*

*FAA Airport Identifier: 18 records (14 do not link to Airports; 4 have no distance information)*

*Reporting Facility used as Identifier: 95 records (no link to Airports or NAVAID Systems)*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*FAA Airport Identifier: 347 records*

*NAVAID System: 10 records*




## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

In [2]:
output471 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace_All_Points_Standard_471OUTOF1543.csv')

In [3]:
#output471

In [4]:
output471_df = output471[['REMARKS', 'DATASET', 'UASLOCATION']]
output471_df

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a large uas 2-5 NM away at h...,NAVAID,21NM SW ANX
1,Aircraft reported an orange quad copter uas 3 ...,NAVAID,26 NM S LTJ
2,Aircraft reported an orange quad copter uas mo...,NAVAID,18NM ESE TNV
3,"Aircraft observed a uas while S bound at 2,000...",DESIGNATED POINT,2 NM N of TUGGZ
4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW
...,...,...,...
466,Aircraft observed a uas while northbound at 2...,FIXED RADIAL,012 NM 030 of AGC
467,Aircraft reported a yellow drone approximately...,FIXED RADIAL,012 NM 036 of DCA
468,Aircraft reported a small yellow drone DCA0660...,FIXED RADIAL,030 NM 066 of DCA
469,DCA-2: Aircraft reported a blue and red uas at...,FIXED RADIAL,008 NM 201 of DCA


In [5]:
# Determine Unique Origin Location References within the 471 record kickback
output471_df['DATASET'].unique()

array(['NAVAID', 'DESIGNATED POINT', 'AIRPORT', 'AIRPORT(?)',
       'FIXED RADIAL'], dtype=object)

# AIRPORTS

In [6]:
output471_airport = output471_df[output471_df['DATASET'] == 'AIRPORT'].reset_index()
output471_airport

,index,REMARKS,DATASET,UASLOCATION
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC
...,...,...,...,...
240,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA
241,245,"Aircraft reported a YAS at 4,500 feet while W ...",AIRPORT,15 NM SE PDZ
242,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL
243,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK


In [7]:
# Export to .csv 
output471_airport.to_csv('airport245.csv', index = False)

In [8]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

In [9]:
test = uas_loc_parts(output471_airport)
test

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC
...,...,...,...,...,...,...,...,...,...
240,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA
241,245,"Aircraft reported a YAS at 4,500 feet while W ...",AIRPORT,15 NM SE PDZ,15,27.780,SE,135,PDZ
242,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL
243,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK


In [10]:
# Export to .csv 
test.to_csv('test245.csv', index = False)

In [11]:
def link_airportsCleaned(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='left')

In [12]:
test1 = link_airportsCleaned(test)
test1

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW,10BA3990-72A1-4F14-89DB-FC7DF04B54F7,Lakefront,30.042417,-90.028250,KNEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC,338AC1EC-7F12-4F3D-932E-E37882849344,Rocky Mountain Metro,39.908806,-105.117194,KBJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY,F0AFB4E6-592A-46B2-A76A-5376CC5A0016,Van Nuys,34.209806,-118.489972,KVNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT,8FBCF8C3-B29D-470D-B4AF-93E8218EDFD5,Bill and Hillary Clinton Ntl/Adams Fld,34.729444,-92.224778,KLIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA,2FD31A12-EF68-4339-891A-D84B47A5EE7B,Dupage,41.907053,-88.247995,KDPA
241,245,"Aircraft reported a YAS at 4,500 feet while W ...",AIRPORT,15 NM SE PDZ,15,27.780,SE,135,PDZ,NaN,NaN,NaN,NaN,NaN
242,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL,48203599-A2E2-467A-BB88-DAC28F3A7FA6,Dallas Love Fld,32.845945,-96.850877,KDAL
243,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK


In [13]:
test1.dropna()

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW,10BA3990-72A1-4F14-89DB-FC7DF04B54F7,Lakefront,30.042417,-90.028250,KNEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC,338AC1EC-7F12-4F3D-932E-E37882849344,Rocky Mountain Metro,39.908806,-105.117194,KBJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY,F0AFB4E6-592A-46B2-A76A-5376CC5A0016,Van Nuys,34.209806,-118.489972,KVNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT,8FBCF8C3-B29D-470D-B4AF-93E8218EDFD5,Bill and Hillary Clinton Ntl/Adams Fld,34.729444,-92.224778,KLIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,243,"Aircraft reported a white fixed wing uas at 2,...",AIRPORT,10 NM SW DFW,10,18.520,SW,225,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
240,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA,2FD31A12-EF68-4339-891A-D84B47A5EE7B,Dupage,41.907053,-88.247995,KDPA
242,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL,48203599-A2E2-467A-BB88-DAC28F3A7FA6,Dallas Love Fld,32.845945,-96.850877,KDAL
243,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK


In [14]:
noMatch_airports = test1[test1['LATITUDE'].isnull()]
len(noMatch_airports)

14

In [15]:
# Export to .csv 
noMatch_airports.to_csv('airports245_nomatch14.csv', index = False)

In [16]:
true_airports = test1[test1['LATITUDE'].notnull()].reset_index(drop=True)
true_airports

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW,10BA3990-72A1-4F14-89DB-FC7DF04B54F7,Lakefront,30.042417,-90.028250,KNEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC,338AC1EC-7F12-4F3D-932E-E37882849344,Rocky Mountain Metro,39.908806,-105.117194,KBJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY,F0AFB4E6-592A-46B2-A76A-5376CC5A0016,Van Nuys,34.209806,-118.489972,KVNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT,8FBCF8C3-B29D-470D-B4AF-93E8218EDFD5,Bill and Hillary Clinton Ntl/Adams Fld,34.729444,-92.224778,KLIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,243,"Aircraft reported a white fixed wing uas at 2,...",AIRPORT,10 NM SW DFW,10,18.520,SW,225,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
227,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA,2FD31A12-EF68-4339-891A-D84B47A5EE7B,Dupage,41.907053,-88.247995,KDPA
228,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL,48203599-A2E2-467A-BB88-DAC28F3A7FA6,Dallas Love Fld,32.845945,-96.850877,KDAL
229,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK


`4 remarks that are missing valid distance information regarding the UAS location`

In [56]:
airports_invalidDist = true_airports[true_airports['Distance_NM'] == 0].reset_index(drop=True)
airports_invalidDist

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,24,Aircraft observed a uas 100 feet high off the ...,AIRPORT,NM SW of APA,0,0.0,SW,225,APA,F45128FD-465B-4CD2-9CCA-F022A4152561,Centennial,39.570119,-104.849293,KAPA
1,28,Aircraft observed a uas off the left side whil...,AIRPORT,NM S of PTW,0,0.0,S,180,PTW,164605B4-1F5A-4259-99DF-F1E81793C5E0,Heritage Fld,40.239556,-75.556722,KPTW
2,126,Aircraft reported a uas 2NM NM E of PAE whileS...,AIRPORT,NM E of PAE,0,0.0,E,90,PAE,32EB01A7-F6A3-4C0F-8238-D92BB8F2B019,Snohomish County (Paine Fld),47.907317,-122.282094,KPAE
3,139,Aircraft reported a silver uas off the left si...,AIRPORT,NM NW of SFO,0,0.0,NW,315,SFO,AF8BC34D-2D11-4CDB-9E87-33A0D430C055,San Francisco Intl,37.618806,-122.375417,KSFO


In [57]:
# Export to .csv 
airports_invalidDist.to_csv('airNoDist04OF471_EXCEPTIONS.csv', index = False)

`227 records with valid distance information regarding UAS location`

In [17]:
airports_validDist = true_airports[true_airports['Distance_NM'] != 0].reset_index(drop=True)
airports_validDist

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW,10BA3990-72A1-4F14-89DB-FC7DF04B54F7,Lakefront,30.042417,-90.028250,KNEW
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC,338AC1EC-7F12-4F3D-932E-E37882849344,Rocky Mountain Metro,39.908806,-105.117194,KBJC
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY,F0AFB4E6-592A-46B2-A76A-5376CC5A0016,Van Nuys,34.209806,-118.489972,KVNY
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT,8FBCF8C3-B29D-470D-B4AF-93E8218EDFD5,Bill and Hillary Clinton Ntl/Adams Fld,34.729444,-92.224778,KLIT
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,243,"Aircraft reported a white fixed wing uas at 2,...",AIRPORT,10 NM SW DFW,10,18.520,SW,225,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
223,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA,2FD31A12-EF68-4339-891A-D84B47A5EE7B,Dupage,41.907053,-88.247995,KDPA
224,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL,48203599-A2E2-467A-BB88-DAC28F3A7FA6,Dallas Love Fld,32.845945,-96.850877,KDAL
225,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK


In [18]:
# Export to .csv 
airports_validDist.to_csv('airports245_validDist227.csv', index = False)

In [19]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe

In [20]:
air227 = uas_lat_long(airports_validDist)
air227

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,4,Aircraft observed a 20-30 pound uas pass withi...,AIRPORT,1NM SE NEW,1,1.852,SE,135,NEW,10BA3990-72A1-4F14-89DB-FC7DF04B54F7,Lakefront,30.042417,-90.028250,KNEW,30.030602,-90.014673
1,5,Aircraft observed a quad copter uas 500 below ...,AIRPORT,2 NM SW BJC,2,3.704,SW,225,BJC,338AC1EC-7F12-4F3D-932E-E37882849344,Rocky Mountain Metro,39.908806,-105.117194,KBJC,39.885213,-105.147814
2,6,Aircraft observed a silver 4 rotor wing uas at...,AIRPORT,10 NM NNW VNY,10,18.520,NNW,338,VNY,F0AFB4E6-592A-46B2-A76A-5376CC5A0016,Van Nuys,34.209806,-118.489972,KVNY,34.364581,-118.565392
3,7,Aircraft observed a medium sized quadcopter ua...,AIRPORT,3NM N LIT,3,5.556,N,0,LIT,8FBCF8C3-B29D-470D-B4AF-93E8218EDFD5,Bill and Hillary Clinton Ntl/Adams Fld,34.729444,-92.224778,KLIT,34.779527,-92.224778
4,8,Aircraft observed a white quad copter type uas...,AIRPORT,4.5 NM W SLC,4.5,8.334,W,270,SLC,CB41ECB2-ADC7-49A5-9C05-81854AFD415C,Salt Lake City Intl,40.788388,-111.977773,KSLC,40.788346,-112.076513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,243,"Aircraft reported a white fixed wing uas at 2,...",AIRPORT,10 NM SW DFW,10,18.520,SW,225,DFW,B371E693-B91F-40ED-ABB1-628EFE9D3111,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,32.779072,-97.177478
223,244,Aircraft reported a white uas off the left sid...,AIRPORT,3 NM W of DPA,3,5.556,W,270,DPA,2FD31A12-EF68-4339-891A-D84B47A5EE7B,Dupage,41.907053,-88.247995,KDPA,41.907034,-88.314958
224,246,Aircraft reported a quadcopter uas 500 feet be...,AIRPORT,7 NM ENE DAL,7,12.964,ENE,68,DAL,48203599-A2E2-467A-BB88-DAC28F3A7FA6,Dallas Love Fld,32.845945,-96.850877,KDAL,32.889669,-96.722416
225,247,"Aircraft reported a uas while SE at 10,000 fee...",AIRPORT,20 NM SE JFK,20,37.040,SE,135,JFK,DD5FB154-0787-4B0F-8554-C033FAFA0155,John F Kennedy Intl,40.639928,-73.778693,KJFK,40.403653,-73.470157


In [21]:
# Export to .csv 
air227.to_csv('airport227_uasLatLong.csv', index = False)

# AIRPORT(?)
`Using the REPORTINGFACILITY field here, so the identifier may be an airport, or may be a navaid or TRACON, etc.`

In [22]:
output471_airportQ = output471_df[output471_df['DATASET'] == 'AIRPORT(?)'].reset_index()
output471_airportQ

,index,REMARKS,DATASET,UASLOCATION
0,249,Aircraft observed a uas operating 400 feet bel...,AIRPORT(?),1.5 NM NW of POC
1,250,Aircraft observed a uas at the same altitude w...,AIRPORT(?),5NM E of NMM
2,251,"Aircraft observed a uas while SSW bound at 1,4...",AIRPORT(?),1 NM NE of SDL
3,252,"Aircraft observed a red uas hovering at 1,000 ...",AIRPORT(?),5NM S of D01
4,253,"Aircraft observed a black uas, one foot in dia...",AIRPORT(?),5 NM NW of CLE
...,...,...,...,...
189,438,Aircraft reported a gray uas while NE bound at...,AIRPORT(?),5 NM SE N90
190,439,Aircraft reported a large black fixed wing bla...,AIRPORT(?),5 NM E I90
191,440,Reporting facility unreported. Aircraft report...,AIRPORT(?),10 NM NE of SW1
192,441,Aircraft reported a uas or rocket with a parac...,AIRPORT(?),7 NM SSW A90


In [23]:
test_airQ = uas_loc_parts(output471_airportQ)
test_airQ

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,249,Aircraft observed a uas operating 400 feet bel...,AIRPORT(?),1.5 NM NW of POC,1.5,2.778,NW,315,POC
1,250,Aircraft observed a uas at the same altitude w...,AIRPORT(?),5NM E of NMM,5,9.260,E,90,NMM
2,251,"Aircraft observed a uas while SSW bound at 1,4...",AIRPORT(?),1 NM NE of SDL,1,1.852,NE,45,SDL
3,252,"Aircraft observed a red uas hovering at 1,000 ...",AIRPORT(?),5NM S of D01,5,9.260,S,180,D01
4,253,"Aircraft observed a black uas, one foot in dia...",AIRPORT(?),5 NM NW of CLE,5,9.260,NW,315,CLE
...,...,...,...,...,...,...,...,...,...
189,438,Aircraft reported a gray uas while NE bound at...,AIRPORT(?),5 NM SE N90,5,9.260,SE,135,N90
190,439,Aircraft reported a large black fixed wing bla...,AIRPORT(?),5 NM E I90,5,9.260,E,90,I90
191,440,Reporting facility unreported. Aircraft report...,AIRPORT(?),10 NM NE of SW1,10,18.520,NE,45,SW1
192,441,Aircraft reported a uas or rocket with a parac...,AIRPORT(?),7 NM SSW A90,7,12.964,SSW,203,A90


In [24]:
airq_airlink1 = link_airportsCleaned(test_airQ)
airq_airlink1

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,249,Aircraft observed a uas operating 400 feet bel...,AIRPORT(?),1.5 NM NW of POC,1.5,2.778,NW,315,POC,C16DBC19-4892-4450-B04D-F25868853F9E,Brackett Fld,34.091667,-117.781778,KPOC
1,250,Aircraft observed a uas at the same altitude w...,AIRPORT(?),5NM E of NMM,5,9.260,E,90,NMM,D4D40016-A8BC-40D8-A81A-AC03395037FE,Meridian NAS /Mc Cain Fld/,32.553481,-88.555264,KNMM
2,251,"Aircraft observed a uas while SSW bound at 1,4...",AIRPORT(?),1 NM NE of SDL,1,1.852,NE,45,SDL,73A4E33E-FACF-472B-860B-37A8BACCDC2C,Scottsdale,33.622889,-111.910528,KSDL
3,252,"Aircraft observed a red uas hovering at 1,000 ...",AIRPORT(?),5NM S of D01,5,9.260,S,180,D01,NaN,NaN,NaN,NaN,NaN
4,253,"Aircraft observed a black uas, one foot in dia...",AIRPORT(?),5 NM NW of CLE,5,9.260,NW,315,CLE,2A962A82-442A-4A82-9535-214B1B978E2E,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,438,Aircraft reported a gray uas while NE bound at...,AIRPORT(?),5 NM SE N90,5,9.260,SE,135,N90,NaN,NaN,NaN,NaN,NaN
190,439,Aircraft reported a large black fixed wing bla...,AIRPORT(?),5 NM E I90,5,9.260,E,90,I90,NaN,NaN,NaN,NaN,NaN
191,440,Reporting facility unreported. Aircraft report...,AIRPORT(?),10 NM NE of SW1,10,18.520,NE,45,SW1,NaN,NaN,NaN,NaN,NaN
192,441,Aircraft reported a uas or rocket with a parac...,AIRPORT(?),7 NM SSW A90,7,12.964,SSW,203,A90,NaN,NaN,NaN,NaN,NaN


In [25]:
airq_airlink = airq_airlink1[airq_airlink1['LATITUDE'].notnull()].reset_index(drop=True)
airq_airlink

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,249,Aircraft observed a uas operating 400 feet bel...,AIRPORT(?),1.5 NM NW of POC,1.5,2.778,NW,315,POC,C16DBC19-4892-4450-B04D-F25868853F9E,Brackett Fld,34.091667,-117.781778,KPOC
1,250,Aircraft observed a uas at the same altitude w...,AIRPORT(?),5NM E of NMM,5,9.260,E,90,NMM,D4D40016-A8BC-40D8-A81A-AC03395037FE,Meridian NAS /Mc Cain Fld/,32.553481,-88.555264,KNMM
2,251,"Aircraft observed a uas while SSW bound at 1,4...",AIRPORT(?),1 NM NE of SDL,1,1.852,NE,45,SDL,73A4E33E-FACF-472B-860B-37A8BACCDC2C,Scottsdale,33.622889,-111.910528,KSDL
3,253,"Aircraft observed a black uas, one foot in dia...",AIRPORT(?),5 NM NW of CLE,5,9.260,NW,315,CLE,2A962A82-442A-4A82-9535-214B1B978E2E,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE
4,255,Aircraft observed a blue uas at the same altit...,AIRPORT(?),3 NM S of JAX,3,5.556,S,180,JAX,E2E20781-927D-4930-B08E-27A9C0FE0984,Jacksonville Intl,30.494046,-81.687847,KJAX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,428,Aircraft observed a uas off the front side whi...,AIRPORT(?),6 NM SW GSB,6,11.112,SW,225,GSB,B09BE7CE-8921-4760-8DF3-B7226DD0EE65,Seymour Johnson AFB,35.339385,-77.960594,KGSB
95,430,"Aircraft observed a uas while E bound at 7,000...",AIRPORT(?),5 NM W of PVD,5,9.260,W,270,PVD,3F706D32-8D17-40DB-8A06-71C13F2158F1,Theodore Francis Green State,41.722333,-71.427722,KPVD
96,433,Aircraft reported a red uas 50 feet below thei...,AIRPORT(?),8 NM E of SJC,8,14.816,E,90,SJC,ACA1CDD6-3606-47F8-9A7B-904DB1B3DBC5,Norman Y Mineta San Jose Intl,37.362995,-121.928621,KSJC
97,434,Aircraft reported a uas18 NW of PSPwhileESE bo...,AIRPORT(?),18 NM NW of PSP,18,33.336,NW,315,PSP,D207DB5C-CC14-49EC-BFC7-9D5BFC1F9979,Palm Springs Intl,33.829667,-116.506694,KPSP


In [26]:
air99 = uas_lat_long(airq_airlink)
air99

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,249,Aircraft observed a uas operating 400 feet bel...,AIRPORT(?),1.5 NM NW of POC,1.5,2.778,NW,315,POC,C16DBC19-4892-4450-B04D-F25868853F9E,Brackett Fld,34.091667,-117.781778,KPOC,34.109374,-117.803068
1,250,Aircraft observed a uas at the same altitude w...,AIRPORT(?),5NM E of NMM,5,9.260,E,90,NMM,D4D40016-A8BC-40D8-A81A-AC03395037FE,Meridian NAS /Mc Cain Fld/,32.553481,-88.555264,KNMM,32.553442,-88.456671
2,251,"Aircraft observed a uas while SSW bound at 1,4...",AIRPORT(?),1 NM NE of SDL,1,1.852,NE,45,SDL,73A4E33E-FACF-472B-860B-37A8BACCDC2C,Scottsdale,33.622889,-111.910528,KSDL,33.634695,-111.896413
3,253,"Aircraft observed a black uas, one foot in dia...",AIRPORT(?),5 NM NW of CLE,5,9.260,NW,315,CLE,2A962A82-442A-4A82-9535-214B1B978E2E,Cleveland-Hopkins Intl,41.409407,-81.854691,KCLE,41.468336,-81.933073
4,255,Aircraft observed a blue uas at the same altit...,AIRPORT(?),3 NM S of JAX,3,5.556,S,180,JAX,E2E20781-927D-4930-B08E-27A9C0FE0984,Jacksonville Intl,30.494046,-81.687847,KJAX,30.443928,-81.687847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,428,Aircraft observed a uas off the front side whi...,AIRPORT(?),6 NM SW GSB,6,11.112,SW,225,GSB,B09BE7CE-8921-4760-8DF3-B7226DD0EE65,Seymour Johnson AFB,35.339385,-77.960594,KGSB,35.268532,-78.046950
95,430,"Aircraft observed a uas while E bound at 7,000...",AIRPORT(?),5 NM W of PVD,5,9.260,W,270,PVD,3F706D32-8D17-40DB-8A06-71C13F2158F1,Theodore Francis Green State,41.722333,-71.427722,KPVD,41.722279,-71.539007
96,433,Aircraft reported a red uas 50 feet below thei...,AIRPORT(?),8 NM E of SJC,8,14.816,E,90,SJC,ACA1CDD6-3606-47F8-9A7B-904DB1B3DBC5,Norman Y Mineta San Jose Intl,37.362995,-121.928621,KSJC,37.362876,-121.761372
97,434,Aircraft reported a uas18 NW of PSPwhileESE bo...,AIRPORT(?),18 NM NW of PSP,18,33.336,NW,315,PSP,D207DB5C-CC14-49EC-BFC7-9D5BFC1F9979,Palm Springs Intl,33.829667,-116.506694,KPSP,34.041915,-116.761971


In [27]:
# Export to .csv 
air99.to_csv('air99_uasLatLong.csv', index = False)

In [28]:
airq_airNOlink = airq_airlink1[airq_airlink1['LATITUDE'].isnull()].reset_index(drop=True)
airq_airNOlink

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,252,"Aircraft observed a red uas hovering at 1,000 ...",AIRPORT(?),5NM S of D01,5,9.260,S,180,D01,NaN,NaN,NaN,NaN,NaN
1,254,Aircraft observed a uas off their left side wh...,AIRPORT(?),10NM ENE of F11,10,18.520,ENE,68,F11,NaN,NaN,NaN,NaN,NaN
2,256,Aircraft observed a uas 700 feet below while w...,AIRPORT(?),5 NM NW of ZME,5,9.260,NW,315,ZME,NaN,NaN,NaN,NaN,NaN
3,257,"Aircraft observed a uas E bound at 9,300 feet ...",AIRPORT(?),5 NM NW of M03,5,9.260,NW,315,M03,NaN,NaN,NaN,NaN,NaN
4,258,Aircraft observed a silver uas while NE bound ...,AIRPORT(?),1NM S of WOC,1,1.852,S,180,WOC,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,437,"Aircraft reported a silver quad copter at 6,50...",AIRPORT(?),9 NM NW of C90,9,16.668,NW,315,C90,NaN,NaN,NaN,NaN,NaN
91,438,Aircraft reported a gray uas while NE bound at...,AIRPORT(?),5 NM SE N90,5,9.260,SE,135,N90,NaN,NaN,NaN,NaN,NaN
92,439,Aircraft reported a large black fixed wing bla...,AIRPORT(?),5 NM E I90,5,9.260,E,90,I90,NaN,NaN,NaN,NaN,NaN
93,440,Reporting facility unreported. Aircraft report...,AIRPORT(?),10 NM NE of SW1,10,18.520,NE,45,SW1,NaN,NaN,NaN,NaN,NaN


In [53]:
noair_nonav = pd.merge(airq_airNOlink, output471, on = 'REMARKS', how = 'left')
noair_nonav

,index_x,REMARKS,DATASET_x,UASLOCATION_x,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,...,UASWEIGHTGT55.CEDAR,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET_y,UASLOCATION_y,RWYLOCATION
0,252,"Aircraft observed a red uas hovering at 1,000 ...",AIRPORT(?),5NM S of D01,5,9.260,S,180,D01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5NM S of D01,NaN
1,254,Aircraft observed a uas off their left side wh...,AIRPORT(?),10NM ENE of F11,10,18.520,ENE,68,F11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),10NM ENE of F11,NaN
2,256,Aircraft observed a uas 700 feet below while w...,AIRPORT(?),5 NM NW of ZME,5,9.260,NW,315,ZME,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5 NM NW of ZME,NaN
3,257,"Aircraft observed a uas E bound at 9,300 feet ...",AIRPORT(?),5 NM NW of M03,5,9.260,NW,315,M03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5 NM NW of M03,NaN
4,258,Aircraft observed a silver uas while NE bound ...,AIRPORT(?),1NM S of WOC,1,1.852,S,180,WOC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),1NM S of WOC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,437,"Aircraft reported a silver quad copter at 6,50...",AIRPORT(?),9 NM NW of C90,9,16.668,NW,315,C90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),9 NM NW of C90,NaN
91,438,Aircraft reported a gray uas while NE bound at...,AIRPORT(?),5 NM SE N90,5,9.260,SE,135,N90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5 NM SE N90,NaN
92,439,Aircraft reported a large black fixed wing bla...,AIRPORT(?),5 NM E I90,5,9.260,E,90,I90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5 NM E I90,NaN
93,440,Reporting facility unreported. Aircraft report...,AIRPORT(?),10 NM NE of SW1,10,18.520,NE,45,SW1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),10 NM NE of SW1,NaN


In [55]:
# Export to .csv 
noair_nonav.to_csv('noAirNav95OF471_EXCEPTIONS.csv', index = False)

In [29]:
#test_airnav = link_navaidCleaned(airq_airNOlink)
#test_airnav

# NAVAID Systems

In [30]:
output471_nav = output471_df[output471_df['DATASET'] == 'NAVAID'].reset_index()
output471_nav

,index,REMARKS,DATASET,UASLOCATION
0,0,Aircraft observed a large uas 2-5 NM away at h...,NAVAID,21NM SW ANX
1,1,Aircraft reported an orange quad copter uas 3 ...,NAVAID,26 NM S LTJ
2,2,Aircraft reported an orange quad copter uas mo...,NAVAID,18NM ESE TNV


In [31]:
test_nav = uas_loc_parts(output471_nav)
test_nav

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,0,Aircraft observed a large uas 2-5 NM away at h...,NAVAID,21NM SW ANX,21,38.892,SW,225,ANX
1,1,Aircraft reported an orange quad copter uas 3 ...,NAVAID,26 NM S LTJ,26,48.152,S,180,LTJ
2,2,Aircraft reported an orange quad copter uas mo...,NAVAID,18NM ESE TNV,18,33.336,ESE,113,TNV


In [32]:
def link_navaidCleaned(dataframe):
    '''
    This function links the provided dataframe to the NAVAID_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with navaid lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    navC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    return pd.merge(dataframe, navC, on='IDENT', how='left')

In [33]:
link_navaidCleaned(test_nav)

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,0,Aircraft observed a large uas 2-5 NM away at h...,NAVAID,21NM SW ANX,21,38.892,SW,225,ANX,-94.128842,39.095431,NAPOLEON,NAPOLEON
1,1,Aircraft reported an orange quad copter uas 3 ...,NAVAID,26 NM S LTJ,26,48.152,S,180,LTJ,-121.100848,45.713589,KLICKITAT,THE DALLES
2,2,Aircraft reported an orange quad copter uas mo...,NAVAID,18NM ESE TNV,18,33.336,ESE,113,TNV,-96.058247,30.288532,NAVASOTA,NAVASOTA


In [34]:
nav3_latLong = uas_lat_long(link_navaidCleaned(test_nav))
nav3_latLong

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,0,Aircraft observed a large uas 2-5 NM away at h...,NAVAID,21NM SW ANX,21,38.892,SW,225,ANX,-94.128842,39.095431,NAPOLEON,NAPOLEON,38.847278,-94.445626
1,1,Aircraft reported an orange quad copter uas 3 ...,NAVAID,26 NM S LTJ,26,48.152,S,180,LTJ,-121.100848,45.713589,KLICKITAT,THE DALLES,45.280339,-121.100848
2,2,Aircraft reported an orange quad copter uas mo...,NAVAID,18NM ESE TNV,18,33.336,ESE,113,TNV,-96.058247,30.288532,NAVASOTA,NAVASOTA,30.170646,-95.739666


In [35]:
# Export to .csv 
nav3_latLong.to_csv('nav3_uasLatLong.csv', index = False)

# DESIGNATED POINTS

In [36]:
output471_dp = output471_df[output471_df['DATASET'] == 'DESIGNATED POINT'].reset_index()
output471_dp

,index,REMARKS,DATASET,UASLOCATION
0,3,"Aircraft observed a uas while S bound at 2,000...",DESIGNATED POINT,2 NM N of TUGGZ


In [37]:
uas_loc_parts(output471_dp)

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,3,"Aircraft observed a uas while S bound at 2,000...",DESIGNATED POINT,2 NM N of TUGGZ,2,3.704,N,0,TUGGZ


In [38]:
def dp_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    dp_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/designated_points_cleaned.csv')
    
    output_df = pd.merge(dataframe, dp_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

In [39]:
dp_link(uas_loc_parts(output471_dp))

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,Unnamed: 0,LATITUDE,LONGITUDE
0,3,"Aircraft observed a uas while S bound at 2,000...",DESIGNATED POINT,2 NM N of TUGGZ,2,3.704,N,0,TUGGZ,NaN,NaN,NaN


In [40]:
# Export to .csv 
output471_dp.to_csv('dp1OF471_EXCEPTIONS.csv', index = False)

# FIXED RADIAL DISTANCES

In [41]:
output471_fr = output471_df[output471_df['DATASET'] == 'FIXED RADIAL'].reset_index()
output471_fr

,index,REMARKS,DATASET,UASLOCATION
0,443,"Aircraft observed a uas south bound at 2,500 f...",FIXED RADIAL,009 NM 295 of RIC
1,444,DCA-1: Aircraft observed a uas NW of the DC Ch...,FIXED RADIAL,004 NM 024 of DCA
2,445,NEMAC: Aircraft observed a quad-copter uas wit...,FIXED RADIAL,010 NM 300 of VCV
3,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML
4,447,Aircraft observed a uas at approximately the B...,FIXED RADIAL,025 NM 171 of BWI
5,448,Aircraft observed a uas off the DCA205017 alti...,FIXED RADIAL,017 NM 205 of DCA
6,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC
7,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX
8,451,Aircraft observed a blue uas overhead while W ...,FIXED RADIAL,009 NM 226 of CLT
9,452,Aircraft observed a uas off the left side whil...,FIXED RADIAL,006 NM 080 of RDU


In [42]:
test_fr = uas_loc_parts(output471_fr)
test_fr

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,443,"Aircraft observed a uas south bound at 2,500 f...",FIXED RADIAL,009 NM 295 of RIC,009,16.668,295,295,RIC
1,444,DCA-1: Aircraft observed a uas NW of the DC Ch...,FIXED RADIAL,004 NM 024 of DCA,004,7.408,024,024,DCA
2,445,NEMAC: Aircraft observed a quad-copter uas wit...,FIXED RADIAL,010 NM 300 of VCV,010,18.520,300,300,VCV
3,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML,008,14.816,140,140,AML
4,447,Aircraft observed a uas at approximately the B...,FIXED RADIAL,025 NM 171 of BWI,025,46.300,171,171,BWI
5,448,Aircraft observed a uas off the DCA205017 alti...,FIXED RADIAL,017 NM 205 of DCA,017,31.484,205,205,DCA
6,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC,037,68.524,107,107,SXC
7,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX,015,27.780,142,142,SAX
8,451,Aircraft observed a blue uas overhead while W ...,FIXED RADIAL,009 NM 226 of CLT,009,16.668,226,226,CLT
9,452,Aircraft observed a uas off the left side whil...,FIXED RADIAL,006 NM 080 of RDU,006,11.112,080,080,RDU


In [43]:
fr_airlink1 = link_airportsCleaned(test_fr)
fr_airlink1

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,443,"Aircraft observed a uas south bound at 2,500 f...",FIXED RADIAL,009 NM 295 of RIC,009,16.668,295,295,RIC,69E61D8F-915F-4B1D-9666-6338F1F19CD9,Richmond Intl,37.505181,-77.319739,KRIC
1,444,DCA-1: Aircraft observed a uas NW of the DC Ch...,FIXED RADIAL,004 NM 024 of DCA,004,7.408,024,024,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,445,NEMAC: Aircraft observed a quad-copter uas wit...,FIXED RADIAL,010 NM 300 of VCV,010,18.520,300,300,VCV,D0BF5C97-F023-4A00-A9DE-B7317305C810,Southern California Logistics,34.597472,-117.383000,KVCV
3,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML,008,14.816,140,140,AML,NaN,NaN,NaN,NaN,NaN
4,447,Aircraft observed a uas at approximately the B...,FIXED RADIAL,025 NM 171 of BWI,025,46.300,171,171,BWI,751671DD-2768-454F-AA80-2567DB9933FE,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
5,448,Aircraft observed a uas off the DCA205017 alti...,FIXED RADIAL,017 NM 205 of DCA,017,31.484,205,205,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
6,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC,037,68.524,107,107,SXC,NaN,NaN,NaN,NaN,NaN
7,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX,015,27.780,142,142,SAX,NaN,NaN,NaN,NaN,NaN
8,451,Aircraft observed a blue uas overhead while W ...,FIXED RADIAL,009 NM 226 of CLT,009,16.668,226,226,CLT,91CC8955-7A21-4843-9C70-CC0903339770,Charlotte/Douglas Intl,35.213750,-80.949056,KCLT
9,452,Aircraft observed a uas off the left side whil...,FIXED RADIAL,006 NM 080 of RDU,006,11.112,080,080,RDU,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,Raleigh-Durham Intl,35.877639,-78.787472,KRDU


In [44]:
fr_airlink = fr_airlink1[fr_airlink1['LATITUDE'].notnull()].reset_index(drop=True)
fr_airlink

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,443,"Aircraft observed a uas south bound at 2,500 f...",FIXED RADIAL,009 NM 295 of RIC,009,16.668,295,295,RIC,69E61D8F-915F-4B1D-9666-6338F1F19CD9,Richmond Intl,37.505181,-77.319739,KRIC
1,444,DCA-1: Aircraft observed a uas NW of the DC Ch...,FIXED RADIAL,004 NM 024 of DCA,004,7.408,024,024,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,445,NEMAC: Aircraft observed a quad-copter uas wit...,FIXED RADIAL,010 NM 300 of VCV,010,18.520,300,300,VCV,D0BF5C97-F023-4A00-A9DE-B7317305C810,Southern California Logistics,34.597472,-117.383000,KVCV
3,447,Aircraft observed a uas at approximately the B...,FIXED RADIAL,025 NM 171 of BWI,025,46.300,171,171,BWI,751671DD-2768-454F-AA80-2567DB9933FE,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
4,448,Aircraft observed a uas off the DCA205017 alti...,FIXED RADIAL,017 NM 205 of DCA,017,31.484,205,205,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
5,451,Aircraft observed a blue uas overhead while W ...,FIXED RADIAL,009 NM 226 of CLT,009,16.668,226,226,CLT,91CC8955-7A21-4843-9C70-CC0903339770,Charlotte/Douglas Intl,35.213750,-80.949056,KCLT
6,452,Aircraft observed a uas off the left side whil...,FIXED RADIAL,006 NM 080 of RDU,006,11.112,080,080,RDU,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,Raleigh-Durham Intl,35.877639,-78.787472,KRDU
7,453,Aircraft observed a uas at 600 feet off the le...,FIXED RADIAL,007 NM 023 of BWI,007,12.964,023,023,BWI,751671DD-2768-454F-AA80-2567DB9933FE,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
8,454,Aircraft observed a uas off the right side whi...,FIXED RADIAL,038 NM 297 of DCA,038,70.376,297,297,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
9,455,DCA-1: Aircraft observed a uas off the front s...,FIXED RADIAL,010 NM 038 of DCA,010,18.520,038,038,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [45]:
fix_radial21 = uas_lat_long(fr_airlink)
fix_radial21

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,443,"Aircraft observed a uas south bound at 2,500 f...",FIXED RADIAL,009 NM 295 of RIC,009,16.668,295,295,RIC,69E61D8F-915F-4B1D-9666-6338F1F19CD9,Richmond Intl,37.505181,-77.319739,KRIC,37.568526,-77.490732
1,444,DCA-1: Aircraft observed a uas NW of the DC Ch...,FIXED RADIAL,004 NM 024 of DCA,004,7.408,024,024,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.912401,-77.002982
2,445,NEMAC: Aircraft observed a quad-copter uas wit...,FIXED RADIAL,010 NM 300 of VCV,010,18.520,300,300,VCV,D0BF5C97-F023-4A00-A9DE-B7317305C810,Southern California Logistics,34.597472,-117.383000,KVCV,34.680820,-117.558017
3,447,Aircraft observed a uas at approximately the B...,FIXED RADIAL,025 NM 171 of BWI,025,46.300,171,171,BWI,751671DD-2768-454F-AA80-2567DB9933FE,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,38.763772,-76.585657
4,448,Aircraft observed a uas off the DCA205017 alti...,FIXED RADIAL,017 NM 205 of DCA,017,31.484,205,205,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.594303,-77.190452
5,451,Aircraft observed a blue uas overhead while W ...,FIXED RADIAL,009 NM 226 of CLT,009,16.668,226,226,CLT,91CC8955-7A21-4843-9C70-CC0903339770,Charlotte/Douglas Intl,35.213750,-80.949056,KCLT,35.109314,-81.080573
6,452,Aircraft observed a uas off the left side whil...,FIXED RADIAL,006 NM 080 of RDU,006,11.112,080,080,RDU,CB0BC5B9-0B40-44DE-96E3-5B8E2AEA35C4,Raleigh-Durham Intl,35.877639,-78.787472,KRDU,35.894968,-78.666263
7,453,Aircraft observed a uas at 600 feet off the le...,FIXED RADIAL,007 NM 023 of BWI,007,12.964,023,023,BWI,751671DD-2768-454F-AA80-2567DB9933FE,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,39.283203,-76.610282
8,454,Aircraft observed a uas off the right side whi...,FIXED RADIAL,038 NM 297 of DCA,038,70.376,297,297,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,39.136996,-77.762989
9,455,DCA-1: Aircraft observed a uas off the front s...,FIXED RADIAL,010 NM 038 of DCA,010,18.520,038,038,DCA,96204F8C-A545-4DF9-943D-5472C5BC1697,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.982831,-76.906131


In [46]:
# Export to .csv 
fix_radial21.to_csv('fixRadial21_uasLatLong.csv', index = False)

In [47]:
fr_airNONlink = fr_airlink1[fr_airlink1['LATITUDE'].isnull()].reset_index(drop=True).drop(columns = ['GLOBAL_ID', 'NAME', 'LATITUDE', 'LONGITUDE', 'ICAO_ID'])
fr_airNONlink

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML,008,14.816,140,140,AML
1,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC,037,68.524,107,107,SXC
2,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX,015,27.780,142,142,SAX
3,458,"Aircraft observed a uas E bound at 16,000 feet...",FIXED RADIAL,046 NM 248 of GBN,046,85.192,248,248,GBN
4,459,Aircraft observed a uas while E bound at FL240...,FIXED RADIAL,051 NM 022 of GBN,051,94.452,022,022,GBN
5,465,Aircraft observed a white quadcopter uas off t...,FIXED RADIAL,005 NM 120 of BAL,005,9.260,120,120,BAL
6,470,Aircraft reported a white quad copter at the B...,FIXED RADIAL,010 NM 320 of BRV,010,18.520,320,320,BRV


In [48]:
fr7_nav = link_navaidCleaned(fr_airNONlink)
fr7_nav

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML,008,14.816,140,140,AML,-77.466698,38.934604,ARMEL,HERNDON
1,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC,037,68.524,107,107,SXC,-118.419922,33.375061,SANTA CATALINA,SANTA CATALINA
2,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX,015,27.780,142,142,SAX,-74.538314,41.067552,SPARTA,SPARTA
3,458,"Aircraft observed a uas E bound at 16,000 feet...",FIXED RADIAL,046 NM 248 of GBN,046,85.192,248,248,GBN,-112.674283,32.956263,GILA BEND,GILA BEND
4,459,Aircraft observed a uas while E bound at FL240...,FIXED RADIAL,051 NM 022 of GBN,051,94.452,022,022,GBN,-112.674283,32.956263,GILA BEND,GILA BEND
5,465,Aircraft observed a white quadcopter uas off t...,FIXED RADIAL,005 NM 120 of BAL,005,9.260,120,120,BAL,-76.661258,39.171071,BALTIMORE,BALTIMORE
6,470,Aircraft reported a white quad copter at the B...,FIXED RADIAL,010 NM 320 of BRV,010,18.520,320,320,BRV,-77.352868,38.336265,BROOKE,BROOKE


In [49]:
fix_radial7 = uas_lat_long(fr7_nav)
fix_radial7

,index,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,446,Aircraft observed a Red and Black uas at the A...,FIXED RADIAL,008 NM 140 of AML,008,14.816,140,140,AML,-77.466698,38.934604,ARMEL,HERNDON,38.832315,-77.357018
1,449,Aircraft observed a uas off the front while NW...,FIXED RADIAL,037 NM 107 of SXC,037,68.524,107,107,SXC,-118.419922,33.375061,SANTA CATALINA,SANTA CATALINA,33.192432,-117.717184
2,450,"Aircraft observed a uas 1,500 feet below them ...",FIXED RADIAL,015 NM 142 of SAX,015,27.780,142,142,SAX,-74.538314,41.067552,SPARTA,SPARTA,40.870252,-74.335431
3,458,"Aircraft observed a uas E bound at 16,000 feet...",FIXED RADIAL,046 NM 248 of GBN,046,85.192,248,248,GBN,-112.674283,32.956263,GILA BEND,GILA BEND,32.665658,-113.516348
4,459,Aircraft observed a uas while E bound at FL240...,FIXED RADIAL,051 NM 022 of GBN,051,94.452,022,022,GBN,-112.674283,32.956263,GILA BEND,GILA BEND,33.745274,-112.292441
5,465,Aircraft observed a white quadcopter uas off t...,FIXED RADIAL,005 NM 120 of BAL,005,9.260,120,120,BAL,-76.661258,39.171071,BALTIMORE,BALTIMORE,39.129330,-76.568515
6,470,Aircraft reported a white quad copter at the B...,FIXED RADIAL,010 NM 320 of BRV,010,18.520,320,320,BRV,-77.352868,38.336265,BROOKE,BROOKE,38.463993,-77.489267


In [50]:
# Export to .csv 
fix_radial7.to_csv('fixRadial7_uasLatLong.csv', index = False)